In [7]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 40.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [10]:
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [11]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

PRODUCTS_CSV = "../data/productos.csv"
COMMENTS_CSV = "../data/comentarios.csv"

productos = pd.read_csv(PRODUCTS_CSV)
comentarios = pd.read_csv(COMMENTS_CSV)

In [5]:
print("Productos:", productos.shape)
print("Comentarios:", comentarios.shape)

print("Columnas productos:", productos.columns.tolist())
print("Columnas comentarios:", comentarios.columns.tolist())

Productos: (10, 6)
Comentarios: (620, 6)
Columnas productos: ['id_producto', 'nombre_producto', 'categoria', 'subcategoria', 'precio', 'descripcion']
Columnas comentarios: ['id_comentario', 'id_cliente', 'id_producto', 'calificacion', 'texto_comentario', 'fecha_comentario']


In [6]:
# Merge (inner) para quedarnos con comentarios que sí tengan producto
df = comentarios.merge(productos, on="id_producto", how="inner")
print("Dataset unido:", df.shape)
df.head()

Dataset unido: (620, 11)


,id_comentario,id_cliente,id_producto,calificacion,texto_comentario,fecha_comentario,nombre_producto,categoria,subcategoria,precio,descripcion
0,1,1,2,5,El Smartphone Nexus 5G es un cambio de juego. ...,2024-05-20,Smartphone Nexus 5G,Electrónica,Smartphones,899.50,El Smartphone Nexus 5G redefine la experiencia...
1,2,2,4,4,La Camiseta Deportiva Ultralight es muy cómoda...,2024-05-21,Camiseta Deportiva Ultralight,Ropa,Camisetas,35.00,Camiseta técnica de alto rendimiento para depo...
2,3,3,1,5,La Laptop Gamer Pro es una bestia de rendimien...,2024-05-22,Laptop Gamer Pro,Electrónica,Portátiles,1899.99,Laptop de alto rendimiento para gaming y creac...
3,4,4,5,3,Las Zapatillas Urbanas Fit tienen un diseño mu...,2024-05-23,Zapatillas Urbanas Fit,Ropa,Calzado,85.75,Calzado deportivo y casual diseñado para el co...
4,5,5,3,4,Los Auriculares Inalámbricos X ofrecen un soni...,2024-05-24,Auriculares Inalámbricos X,Electrónica,Audio,125.00,Auriculares Bluetooth de diadema diseñados par...


In [14]:
# Remover Stopwords y Lematizar en español.
try:
    import spacy
    nlp = spacy.load("es_core_news_sm")
    print("spaCy cargado: usando lematización y stopwords de spaCy.")

    def preprocess_spacy(text):
        doc = nlp(str(text).lower())
        tokens = [t.lemma_ for t in doc if not t.is_stop and t.is_alpha]
        return " ".join(tokens)

    df["nombre_producto_proc"] = df["nombre_producto"].apply(preprocess_spacy)
    df["descripcion_proc"] = df["descripcion"].apply(preprocess_spacy)
    df["comentario_proc"] = df["texto_comentario"].apply(preprocess_spacy)

except Exception as e:
    print("spaCy no disponible o error cargando modelo", e)

# Revisa resultados
df[["nombre_producto", "nombre_producto_proc", "descripcion", "descripcion_proc", "texto_comentario", "comentario_proc"]].head()

spaCy cargado: usando lematización y stopwords de spaCy.


,nombre_producto,nombre_producto_proc,descripcion,descripcion_proc,texto_comentario,comentario_proc
0,Smartphone Nexus 5G,smartphonir nexus g,El Smartphone Nexus 5G redefine la experiencia...,smartphonir nexus g redefinar experiencia móvi...,El Smartphone Nexus 5G es un cambio de juego. ...,smartphonir nexus g cambio juego pantalla oled...
1,Camiseta Deportiva Ultralight,camiseta deportivo ultralight,Camiseta técnica de alto rendimiento para depo...,camiseta técnico alto rendimiento deporte fabr...,La Camiseta Deportiva Ultralight es muy cómoda...,camiseta deportivo ultralight cómodo entrenami...
2,Laptop Gamer Pro,laptop gamer pro,Laptop de alto rendimiento para gaming y creac...,laptop alto rendimiento gaming creación conten...,La Laptop Gamer Pro es una bestia de rendimien...,laptop gamer pro bestia rendimiento superar ex...
3,Zapatillas Urbanas Fit,zapatilla urbano fit,Calzado deportivo y casual diseñado para el co...,calzado deportivo casual diseñado confort diar...,Las Zapatillas Urbanas Fit tienen un diseño mu...,zapatilla urbano fit diseño atractivo resultar...
4,Auriculares Inalámbricos X,auricular inalámbrico x,Auriculares Bluetooth de diadema diseñados par...,auricular bluetooth diadema diseñado audiofi é...,Los Auriculares Inalámbricos X ofrecen un soni...,auricular inalámbrico x ofrecer sonido nítido ...


In [30]:
from sentence_transformers import SentenceTransformer

print("SentenceTransformers disponible. Usando modelo multilingüe.")
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
st_model = SentenceTransformer(model_name)

SentenceTransformers disponible. Usando modelo multilingüe.


In [31]:
import random

productos_unicos = df[["id_producto", "nombre_producto_proc", "descripcion_proc"]].drop_duplicates(subset="id_producto").reset_index(drop=True)

nombres_textos = productos_unicos["nombre_producto_proc"].tolist()
productos_textos = productos_unicos["descripcion_proc"].tolist()
comentarios_textos = df["comentario_proc"].tolist()
todos_textos = nombres_textos + productos_textos + comentarios_textos

# Mostrar 5 elementos aleatorios de la lista combinada
muestra_aleatoria = random.sample(todos_textos, 5)

print("Muestra aleatoria de la lista 'todos_textos':")
for i, texto in enumerate(muestra_aleatoria):
    print(f"{i+1}. {texto}")

Muestra aleatoria de la lista 'todos_textos':
1. olla calidad limpiar fácilmente comida pegar
2. laptop superar expectativa potente pantalla alto resolución teclado cómodo
3. encantar diseño minimalista útil mochilo perfecto universidad puerto carga ventaja
4. cámara aplicación móvil lento calidad video excelente usabilidad
5. laptop potente pantalla fuga luz detalle menor rendimiento fantástico


In [32]:
embeddings = st_model.encode(todos_textos, convert_to_numpy=True, show_progress_bar=True)
embedding_dim = embeddings.shape[1]
print("Embeddings generados con SentenceTransformers - shape:", embeddings.shape)

Batches: 100%|██████████| 20/20 [00:00<00:00, 89.65it/s]

Embeddings generados con SentenceTransformers - shape: (640, 384)


In [40]:
out_dir = "logs/embeddings"
os.makedirs(out_dir, exist_ok=True)

def make_label(text, n_words=5):
    return " ".join(text.split()[:n_words])

# Crear metadata con columnas: tipo, label, texto_completo
metadata = pd.DataFrame({
    "label": [make_label(t, 5) for t in nombres_textos + productos_textos + comentarios_textos],
    "tipo": ["producto"]*len(nombres_textos) + ["producto"]*len(productos_textos) + ["comentario"]*len(comentarios_textos),
    "texto_completo": nombres_textos + productos_textos + comentarios_textos
})

# Guardar como TSV para TensorBoard Projector
metadata_path = os.path.join(out_dir, "metadata.tsv")
metadata.to_csv(metadata_path, sep="\t", index=False)

print("Metadata guardada en:", metadata_path)

# Guardar checkpoint con los embeddings (TensorFlow expects a Variable)
emb_var = tf.Variable(embeddings, name="embeddings")
ckpt = tf.train.Checkpoint(embedding=emb_var)
ckpt.save(os.path.join(out_dir, "embedding.ckpt"))

print("Embeddings guardados en:", out_dir)
print("Ejecuta en terminal: tensorboard --logdir logs/embeddings")

Metadata guardada en: logs/embeddings/metadata.tsv
Embeddings guardados en: logs/embeddings
Ejecuta en terminal: tensorboard --logdir logs/embeddings


In [34]:
# Concatenamos para generar embeddings juntos
todos_textos = nombres_textos + productos_textos + comentarios_textos

# Separar Embeddings
nombres_count = len(nombres_textos)
desc_count = len(productos_textos)
comm_count = len(comentarios_textos)

# Embeddings de nombres y descripciones de productos
nombres_emb = embeddings[:nombres_count]
desc_emb = embeddings[nombres_count:nombres_count+desc_count]

# Combinar embeddings de nombre + descripción por producto
prod_embeddings = (nombres_emb + desc_emb) / 2

# Embeddings de comentarios
comm_embeddings = embeddings[nombres_count+desc_count:]

def top_k_products_for_comment(comment_idx, k=3):
    q = comm_embeddings[comment_idx]
    # normalizar
    qn = q / np.linalg.norm(q)
    Pn = prod_embeddings / np.linalg.norm(prod_embeddings, axis=1, keepdims=True)
    sims = np.dot(Pn, qn)  # (num_productos,)
    topk_idx = sims.argsort()[::-1][:k]
    return [
        (
            idx,
            sims[idx],
            productos_textos[idx],   # descripción
            nombres_textos[idx],     # nombre producto
            df.iloc[idx]["id_producto"] if "id_producto" in df.columns else None
        )
        for idx in topk_idx
    ]

print("Comentario (procesado):", comentarios_textos[10])
print("Top-3 productos similares:")
for idx, score, desc, nombre, pid in top_k_products_for_comment(10, k=3):
    print(f" - idx_prod={idx}, id_producto={pid}, score={score:.4f}, nombre: {nombre}, desc: {desc}")

Comentario (procesado): rendimiento laptop gamer pro excepcional duración batería punto débil tarea básico durar par hora gaming mantener él conectado tiempo limitar portabilidad ideal escritorio llevar él
Top-3 productos similares:
 - idx_prod=2, id_producto=1, score=0.6368, nombre: laptop gamer pro, desc: laptop alto rendimiento gaming creación contenido procesador intel core generación gb ram tarjeta gráfico nvidia rtx pantalla pulgada tasa refresco resolución ofrecer experiencia visual inmersivo chasis aluminio pulido sistema enfriamiento avanzado garantizar rendimiento óptimo sesión juego larga incluir teclado mecánico retroiluminado personalización rgb tecla
 - idx_prod=8, id_producto=9, score=0.3564, nombre: mochilo urbana tech, desc: mochila moderno funcional vida urbano fabricado poliéster repelente agua compartimento principal funda acolchado laptops pulgada incluir bolsillo frontal organizador bolsillo lateral botella agua puerto carga usb integrado power bank incluido corre

In [ ]:
# tensorboard --logdir logs/embeddings